In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
mpl.rcParams['font.family'] = 'serif'
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
data = pd.read_csv(os.path.join(indir_inter, '_F_dist_religion_bch10.csv'))[['iso',
                                                         'bch10',
                                                         'major_religion',
                                                         'district',
                                                         'lit_14']]
data = data[(data.major_religion.isin(['Christian', 'Muslim', 'Traditional'])) & 
            (data.bch10.isin([1950, 1980]))].reset_index(drop=True)
data = data[~((pd.isnull(data.lit_14)))]
data = data.set_index(['iso', 'district', 'major_religion', 'bch10']).unstack().reset_index(drop=False)
data.columns = ['iso', 'district', 'major_religion', '1950', '1980']
data = data.set_index(['iso', 'district', 'major_religion']).unstack()
data.columns = [x[1] + '_' + x[0] for x in data.columns]
data = data.reset_index(drop=False)

In [4]:
ISO = sorted(list(set(data.iso)))

In [5]:
countries = gpd.read_file(os.path.join(indir_prepro, 'africa_countries', 'africa_countries.shp'))
afrregions = gpd.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
afrregions = afrregions[afrregions.iso.isin(ISO)].reset_index(drop=True).copy()
afrregions = pd.merge(afrregions, data, on=['iso', 'district'], how='left')
africa = afrregions[afrregions.geometry!=None].copy(deep=True)

In [6]:
countries['iso'] = countries.iso3
for c in africa.columns:
    if c not in countries.columns:
        countries[c] = np.nan
countries = countries[[c for c in africa.columns]]
countries = countries[~countries.iso.isin(list(set(africa.iso)) + ['ESP', 'PRT', 'FRA', 'YEM', 'SHN', 'ATF', 'REU'])]

In [7]:
africa = pd.concat([africa, countries], axis=0, ignore_index=True)

In [8]:
africa

,iso,district,geometry,Christian_1950,Muslim_1950,Traditional_1950,Christian_1980,Muslim_1980,Traditional_1980
0,BEN,1.0,"POLYGON ((2.40452 11.67795, 2.42524 11.66394, ...",0.145594,0.040917,0.006993,0.161554,0.090679,0.025135
1,BEN,2.0,"POLYGON ((3.19362 10.90465, 3.19200 10.89730, ...",0.191304,0.030263,0.016129,0.205000,0.075529,0.166667
2,BEN,3.0,"POLYGON ((3.25846 11.29882, 3.25769 11.29174, ...",0.200803,0.075439,0.021277,0.191214,0.143840,0.107143
3,BEN,4.0,"POLYGON ((2.84088 12.40599, 2.84306 12.40454, ...",0.500000,0.019097,0.100000,0.256410,0.075248,0.000000
4,BEN,5.0,"POLYGON ((3.08195 11.97610, 3.08603 11.97523, ...",0.603774,0.039832,0.083333,0.530864,0.106777,0.000000
...,...,...,...,...,...,...,...,...,...
2597,ZWE,NaN,"POLYGON ((29.86384 -15.61295, 29.87364 -15.615...",NaN,NaN,NaN,NaN,NaN,NaN
2598,SDN,NaN,"MULTIPOLYGON (((35.64075 23.09098, 35.64022 23...",NaN,NaN,NaN,NaN,NaN,NaN
2599,SDN,NaN,"MULTIPOLYGON (((37.27043 20.76363, 37.26722 20...",NaN,NaN,NaN,NaN,NaN,NaN
2600,SDN,NaN,"MULTIPOLYGON (((38.79701 18.54980, 38.79571 18...",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
missing_kwds = dict(color='white', label='no Data')

titles = ['literacy 1950s', 'literacy 1950s', 'literacy 1950s', 'literacy 1980s', 'literacy 1980s', 'literacy 1980s']
for idvar, variable in enumerate(['Christian_1950', 'Muslim_1950', 'Traditional_1950', 'Christian_1980', 'Muslim_1980', 'Traditional_1980']):
    title = titles[idvar]

    f, ax = plt.subplots(1,1, figsize=(10, 10))
    africa.plot(ax=ax, 
            column=variable,
            scheme='user_defined',
            classification_kwds={'bins':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]},
            cmap = 'YlOrRd',
            edgecolor='k',
            linewidth=0.1,
            legend=True, 
            missing_kwds=missing_kwds)

    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0.15, 0.3, 0.2, 0.2))
    for txt in leg.texts:
        txti = re.sub('\.([0-9])0', r'.\1',  txt.get_text().replace(', ', ' - '))
        txt.set_text(txti)
    leg.set_title(title, prop={'size': 12, 'weight': 'bold'})
    
    f.savefig(outdir + f'/_4_heatmap_litlevel_{variable}.pdf', bbox_inches='tight')
    plt.close(f)